In [8]:
import pandas as pd
import requests
import time

In [9]:
# Load the dataset
file_path = "C:/Users/kaur6/Downloads/Urban Analytics/Odometer Data/illinois_2023/merged_dataset.csv"  # Update this with the actual file path
df = pd.read_csv(file_path)
print(f"Number of rows in the dataset: {df.shape[0]}")

Number of rows in the dataset: 4144055


In [10]:
# Fetch the first 100 VINs from the dataset
first_500_vins = df['vin'][:500].tolist()

In [11]:
import pyodbc
# Database connection
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=manpreets_asus;DATABASE=vPICList_Lite1;Trusted_Connection=yes;')
cursor = conn.cursor()

In [12]:
# List to store results
all_results = []

# Loop through each VIN and fetch results
for vin in first_500_vins:
    cursor.execute("EXEC [dbo].[spVinDecode] @v = ?", vin)
    results = cursor.fetchall()

    if results:
        # Extract column names
        columns = [column[0] for column in cursor.description]

        # Append data with VIN included
        for row in results:
            row_dict = dict(zip(columns, row))
            row_dict["VIN"] = vin  # Add VIN as a separate column
            all_results.append(row_dict)

# Close database connection
cursor.close()
conn.close()

# Convert results to DataFrame
df_results = pd.DataFrame(all_results)

# Check for duplicates before pivoting
df_results = df_results.groupby(["VIN", "Variable"])["Value"].agg(lambda x: "; ".join(x.astype(str))).reset_index()

# Pivot the DataFrame: Convert 'Variable' column into column headers
df_pivot = df_results.pivot(index="VIN", columns="Variable", values="Value")

# Reset column names for clarity
df_pivot.reset_index(inplace=True)

# Save to CSV
output_file = "C:/Users/kaur6/Downloads/Urban Analytics/Odometer Data/illinois_2023/decoded_vins_results.csv"
df_pivot.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")

Results saved to C:/Users/kaur6/Downloads/Urban Analytics/Odometer Data/illinois_2023/decoded_vins_results.csv


In [13]:
import pandas as pd

# Load the datasets
mpg_file = "C:/Users/kaur6/Downloads/Urban Analytics/Odometer Data/illinois_2023/mpg_with_model.csv"  # Update with actual path
vin_file = "C:/Users/kaur6/Downloads/Urban Analytics/Odometer Data/illinois_2023/decoded_vins_results.csv"  # Update with actual path

df_mpg = pd.read_csv(mpg_file)
df_vin = pd.read_csv(vin_file)

# Standardize column names to lowercase for consistent merging
df_vin.rename(columns={"Model": "model"}, inplace=True)

# Merge datasets on 'model' column
df_merged = df_vin.merge(df_mpg, on="model", how="left")

# Select only VIN and combination_mpg columns
df_final = df_merged[["VIN", "combination_mpg"]]

# Save the final dataset
output_file = "C:/Users/kaur6/Downloads/Urban Analytics/Odometer Data/illinois_2023/vin_with_mpg.csv"
df_final.to_csv(output_file, index=False)

print(f"File saved: {output_file}")

File saved: C:/Users/kaur6/Downloads/Urban Analytics/Odometer Data/illinois_2023/vin_with_mpg.csv


In [9]:
vin = '1N4AL21EX9N445210'
year = 2009

cursor.execute("EXEC [dbo].[spVinDecode] @v = ?, @year = ?", vin, year)

result = cursor.fetchall()

for row in result:
    print(row)

cursor.close()
conn.close()

(None, 'Suggested VIN', '', None, None, '', 142, '', None, None, 'SuggestedVIN', 'string', 'Decoding', 'Corrections', None)
(None, 'Error Code', '0', None, None, '', 143, '0', None, None, 'ErrorCode', 'lookup', 'Decoding', 'Corrections', None)
(None, 'Possible Values', '', None, None, '', 144, '', None, None, 'PossibleValues', 'string', 'Decoding', 'Corrections', None)
(None, 'Additional Error Text', None, None, None, '', 156, None, None, None, 'AdditionalErrorText', 'string', 'Decoding', 'Corrections', None)
(None, 'Error Text', '0 - VIN decoded clean. Check Digit (9th position) is correct', None, None, '', 191, '0 - VIN decoded clean. Check Digit (9th position) is correct', None, None, 'ErrorText', 'string', 'Decoding', 'Corrections', None)
(None, 'Vehicle Descriptor', '1N4AL21E*9N', None, None, '', 196, '1N4AL21E*9N', None, None, 'VehicleDescriptor', 'string', 'Decoding', 'Corrections', None)
('General', 'Make', 'NISSAN', 47271, 2679, 'AL2', 26, '478', None, None, 'Make', 'lookup', 